## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/model_fold_1.h5
/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/model_fold_2.h5
/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
#     h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
#     h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
    h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
#     h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768))

    x = GlobalAveragePooling1D()(h10)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.952780,0.704896,0.259593,0.543854,0.509763,0.559454,0.658481,0.724045,0.564734,...,0.912330,0.920080,0.626650,0.976227,0.963708,0.817998,0.035014,0.053891,0.871072,0.926740
1,46,0.874745,0.554075,0.008738,0.768129,0.807633,0.910733,0.578676,0.464352,0.084324,...,0.718693,0.946933,0.631846,0.969569,0.977719,0.872443,0.915599,0.119178,0.077653,0.900211
2,70,0.909684,0.643535,0.027551,0.804990,0.871554,0.936560,0.619667,0.466192,0.127662,...,0.865419,0.935951,0.626380,0.973713,0.974430,0.867427,0.094492,0.060819,0.855920,0.920322
3,132,0.894983,0.444055,0.009147,0.757403,0.723090,0.929111,0.540787,0.412532,0.071075,...,0.689023,0.954373,0.670695,0.970659,0.985031,0.911728,0.792666,0.148602,0.599879,0.902146
4,200,0.923991,0.446967,0.040445,0.802137,0.730495,0.885179,0.681132,0.574267,0.109357,...,0.722030,0.915341,0.621584,0.969381,0.969525,0.836740,0.259820,0.092526,0.704534,0.918932


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.903482,0.607436,0.041572,0.756022,0.791063,0.853589,0.602374,0.489731,0.215750,...,0.797880,0.940094,0.664623,0.968312,0.972659,0.876684,0.505055,0.131330,0.527527,0.917013
std,2812.670060,0.033880,0.124384,0.075140,0.087722,0.108340,0.127206,0.048952,0.088366,0.182930,...,0.077871,0.019413,0.047282,0.008731,0.009593,0.035575,0.320579,0.057686,0.284087,0.018642
min,39.000000,0.781940,0.379638,0.003898,0.275789,0.232645,0.203954,0.506606,0.317745,0.012035,...,0.594875,0.858633,0.518905,0.930502,0.920356,0.746340,0.004791,0.015287,0.029750,0.841723
25%,2572.000000,0.881807,0.494070,0.009889,0.714137,0.742639,0.841274,0.565950,0.428680,0.077983,...,0.730591,0.931405,0.629958,0.963851,0.967886,0.861310,0.150355,0.089527,0.272003,0.905686
50%,5093.000000,0.903399,0.599471,0.015029,0.760355,0.801506,0.898558,0.594096,0.462796,0.152395,...,0.804841,0.943932,0.664191,0.969462,0.974493,0.883224,0.584559,0.135631,0.546588,0.919790
75%,7482.000000,0.927422,0.707777,0.032230,0.812441,0.858258,0.927928,0.632190,0.529961,0.307787,...,0.866316,0.952827,0.696414,0.974037,0.979493,0.901908,0.800237,0.169748,0.784039,0.929844
max,9640.000000,0.971869,0.882138,0.683263,0.943541,0.976784,0.963892,0.766345,0.771154,0.853300,...,0.944682,0.978670,0.791085,0.986124,0.989744,0.957463,0.943658,0.284558,0.990027,0.963385
